In [1]:
import numpy as np
import pandas as pd
import torch
!pip3 install transformers
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import re

In [2]:
train = pd.read_csv('train_df.csv')
dev = pd.read_csv('dev_df.csv')
test = pd.read_csv('test_df.csv')
train.head()

,Tweet,label
0,@liamch88 yeah! :) playing well,0
1,At least I don't have a guy trying to discoura...,0
2,UPLIFT: If you're still discouraged it means y...,0
3,"...at your age, the heyday in the blood is tam...",0
4,i was so embarrassed when she saw us i was lik...,-2


# Loading the Pre-trained BERT model

In [5]:
# Import pre-trained DistilBERT model and tokenizer
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# Import pre-trained BERT model and tokenizer
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [15]:
def tokenized(dataset):
  tokenized = dataset['Tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True))) # This turns every sentence into the list of ids.
  # max_len = 0
  # for i in tokenized.values:
  #   if len(i) > max_len:
  #       max_len = len(i)
  padded = np.array([i + [0]*(71-len(i)) for i in tokenized.values])
  np.array(padded).shape
  return padded

In [16]:
train_features = tokenized(train)
train_features

array([[  101,  1030,  8230, ...,     0,     0,     0],
       [  101,  2012,  2560, ...,     0,     0,     0],
       [  101,  2039, 18412, ...,     0,     0,     0],
       ...,
       [  101, 16360,  4765, ...,     0,     0,     0],
       [  101,  1030,  4811, ...,     0,     0,     0],
       [  101,  1045,  2572, ...,     0,     0,     0]])

In [17]:
dev_features = tokenized(dev)

In [18]:
train_labels = train['label']
dev_labels = dev['label']

In [19]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
dev_predict = lr_clf.predict(dev_features)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([ 1,  1,  0, -2,  0,  0, -2,  3, -2, -2, -2, -3, -2,  1, -2,  0,  1,
        1,  3,  0, -2, -2,  0, -2,  1,  0,  0,  3,  0,  0,  0,  0, -2,  0,
        0,  0,  0,  0,  0,  1,  0,  3, -2,  0,  0,  1, -2, -2,  0, -2,  0,
       -2, -2, -3, -2, -2,  0, -3,  0, -2,  0, -2,  2,  0, -2,  0, -2,  1,
        1, -2, -2,  0, -3,  0, -2,  0,  0, -2, -2,  1,  0,  0, -1,  1, -2,
       -2, -1,  0, -2,  0,  0,  1,  0,  0, -2,  0,  0,  0,  0,  0,  1,  0,
        0,  0, -2,  0,  0,  0,  0,  1,  0,  0,  3,  0, -2,  0,  0, -2,  0,
        0,  0, -2, -2, -2, -2, -2,  0, -3, -2,  0,  0,  0, -3,  0, -3,  0,
        0,  0,  0,  0,  0, -2,  2, -2,  0,  3,  0,  0, -2,  0,  0,  1, -1,
       -2,  1,  0,  0,  0,  1,  0, -2,  0,  0, -2,  0,  0,  0, -3,  1, -2,
       -2,  0,  0,  0,  3,  0,  3,  1,  1,  0,  0,  1,  2,  0,  0,  0, -2,
        0,  0,  0,  0, -3,  3,  0,  1,  0,  1,  0,  0,  0,  0,  0, -2,  0,
        0, -2,  0,  3,  0,  3,  3,  0,  0,  0,  0,  0, -2, -1,  0,  0,  1,
       -2, -2,  3, -1,  1

In [22]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
print('f1 score is', f1_score(dev_predict, dev_labels,average='micro'))

f1 score is 0.20267260579064586
